# User-Defined Functions (CREATE FUNCTION)

This notebook demonstrates user-defined functions in elastic-script - reusable code blocks that return values and can be used in expressions.

## Functions vs Procedures

| Feature | PROCEDURE | FUNCTION |
|---------|-----------|----------|
| Returns value | No (uses OUT params) | Yes (RETURN statement) |
| Use in expressions | No | Yes |
| Side effects | Expected | Discouraged |
| Call syntax | `CALL proc()` | `func()` in expressions |
| Storage | `CREATE PROCEDURE` | `CREATE FUNCTION` |
| Return type | N/A | `RETURNS type` clause |

## Syntax

```sql
CREATE FUNCTION name(params) RETURNS type AS
BEGIN
    -- statements
    RETURN expression;
END FUNCTION
```

## 1. Basic Function

Create a simple function that adds two numbers:

In [ ]:
%%elastic_script

CREATE FUNCTION add_numbers(a NUMBER, b NUMBER) RETURNS NUMBER AS
BEGIN
    RETURN a + b;
END FUNCTION

## 2. Using Functions in Expressions

Functions can be called anywhere an expression is valid:

In [ ]:
%%elastic_script

CREATE PROCEDURE test_add()
BEGIN
    -- Define the function inline for this demo
    FUNCTION add(a NUMBER, b NUMBER) BEGIN RETURN a + b; END FUNCTION;
    
    -- Use in assignment
    DECLARE result NUMBER;
    SET result = add(10, 20);
    PRINT 'Result: ' || result;
    
    -- Use in expression
    PRINT 'Double result: ' || add(result, result);
    
    -- Use in condition
    IF add(5, 5) > 8 THEN
        PRINT '5 + 5 is greater than 8';
    END IF
END PROCEDURE

CALL test_add();

## 3. String Functions

Functions can return any type:

In [ ]:
%%elastic_script

CREATE FUNCTION format_greeting(name STRING, title STRING) RETURNS STRING AS
BEGIN
    RETURN 'Hello, ' || title || ' ' || name || '!';
END FUNCTION

## 4. Functions with Control Flow

Complex logic with IF/ELSE and multiple RETURN points:

In [ ]:
%%elastic_script

CREATE FUNCTION calculate_grade(score NUMBER) RETURNS STRING AS
BEGIN
    IF score >= 90 THEN
        RETURN 'A';
    ELSEIF score >= 80 THEN
        RETURN 'B';
    ELSEIF score >= 70 THEN
        RETURN 'C';
    ELSEIF score >= 60 THEN
        RETURN 'D';
    ELSE
        RETURN 'F';
    END IF
END FUNCTION

## 5. Functions Returning Documents

Build and return complex data structures:

In [ ]:
%%elastic_script

CREATE FUNCTION create_user_profile(name STRING, email STRING, age NUMBER) RETURNS DOCUMENT AS
BEGIN
    RETURN {
        'name': name,
        'email': email,
        'age': age,
        'created_at': CURRENT_TIMESTAMP,
        'is_adult': age >= 18
    };
END FUNCTION

## 6. Functions Returning Arrays

Generate arrays dynamically:

In [ ]:
%%elastic_script

CREATE PROCEDURE test_array_function()
BEGIN
    FUNCTION generate_range(start NUMBER, count NUMBER) 
    BEGIN 
        DECLARE result ARRAY = [];
        DECLARE i NUMBER;
        FOR i IN start..(start + count - 1) LOOP
            SET result = ARRAY_APPEND(result, i);
        END LOOP
        RETURN result; 
    END FUNCTION;
    
    DECLARE nums ARRAY;
    SET nums = generate_range(1, 5);
    PRINT 'Generated: ' || nums;
END PROCEDURE

CALL test_array_function();

## 7. Functions with Error Handling

Use TRY/CATCH inside functions for robust error handling:

In [ ]:
%%elastic_script

CREATE FUNCTION safe_divide(a NUMBER, b NUMBER) RETURNS NUMBER AS
BEGIN
    TRY
        IF b == 0 THEN
            THROW 'Division by zero' WITH CODE 'MATH_ERROR';
        END IF
        RETURN a / b;
    CATCH
        PRINT 'Warning: ' || error['message'] || ', returning 0';
        RETURN 0;
    END TRY
END FUNCTION

## 8. Calling Functions from Functions

Functions can call other functions for composition:

In [ ]:
%%elastic_script

CREATE PROCEDURE test_composition()
BEGIN
    -- Helper functions
    FUNCTION square(x NUMBER) BEGIN RETURN x * x; END FUNCTION;
    FUNCTION double(x NUMBER) BEGIN RETURN x * 2; END FUNCTION;
    
    -- Composed function
    FUNCTION square_then_double(x NUMBER) 
    BEGIN 
        RETURN double(square(x)); 
    END FUNCTION;
    
    PRINT 'square(3) = ' || square(3);
    PRINT 'double(3) = ' || double(3);
    PRINT 'square_then_double(3) = ' || square_then_double(3);
END PROCEDURE

CALL test_composition();

## 9. Deleting Functions

Remove stored functions with DELETE FUNCTION:

In [ ]:
%%elastic_script

DELETE FUNCTION add_numbers;

## Summary

User-defined functions in elastic-script provide:

1. **RETURNS clause** - Explicit return type declaration
2. **Expression usage** - Use functions anywhere expressions are valid
3. **Multiple return points** - Return from any point in the function
4. **All return types** - NUMBER, STRING, BOOLEAN, DATE, ARRAY, DOCUMENT
5. **Composition** - Functions can call other functions
6. **Error handling** - TRY/CATCH works inside functions
7. **Persistence** - CREATE FUNCTION stores for reuse

### Best Practices

- Keep functions focused on a single task
- Use meaningful names that describe what the function returns
- Avoid side effects (no PRINT, no modifying global state)
- Handle errors gracefully with TRY/CATCH
- Use OUT parameters only when you need multiple return values